In [ ]:
# import all necessary packages for CBOW
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import random
import os
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import collections
import itertools
import re
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from nltk.stem import WordNetLemmatizer
from scipy.stats import pearsonr, spearmanr
from torchtext.vocab import GloVe
from sklearn.model_selection import train_test_split

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print device name: get_device_name()
# print(torch.cuda.get_device_name(0))

In [3]:
import io
from gensim.models import KeyedVectors
import gensim.downloader as api
from gensim import matutils
from numpy import dot


print(list(api.info()['models'].keys()))

# def load_vectors(fname):
#     fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
#     n, d = map(int, fin.readline().split())
#     data = {}
#     for line in fin:
#         tokens = line.rstrip().split(' ')
#         data[tokens[0]] = map(float, tokens[1:])
#     return data

# fasttext_data = load_vectors('/content/drive/My Drive/wiki-news-300d-1M.vec')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
model = api.load('fasttext-wiki-news-subwords-300')

In [7]:
def get_word_embedding(model ,word):
    return model[word]

In [8]:
# Cosine similarity between same words
w1 = model['survey']
w2 = model['generation']
print(type(w1))
ans1 = dot(matutils.unitvec(w1), matutils.unitvec(w2))
print(ans1)

ans2 = model.similarity('survey', 'generation')
print(ans2)

<class 'numpy.ndarray'>
0.34138656
0.34138656


In [9]:
import torch
import torch.nn.functional as F

# Assuming 'model' is your pre-loaded word embedding model
x = torch.tensor(model['king'])
y = torch.tensor(model['queen'])
# z = king - man + woman
z = x - torch.tensor(model['man']) + torch.tensor(model['woman'])

# Calculating distances and similarities
distance_king_queen = torch.norm(x - y).item()
cosine_sim_king_queen = F.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0)).item()
distance_king_z = torch.norm(x - z).item()
cosine_sim_king_z = F.cosine_similarity(x.unsqueeze(0), z.unsqueeze(0)).item()

print("Distance between king and queen: ", distance_king_queen)
print("Cosine similarity between king and queen: ", cosine_sim_king_queen)
print("New Distance between king and z: ", distance_king_z)
print("New Cosine similarity between king and z: ", cosine_sim_king_z)


Distance between king and queen:  0.7606731653213501
Cosine similarity between king and queen:  0.7704245448112488
New Distance between king and z:  0.6913657188415527
New Cosine similarity between king and z:  0.8185327053070068


In [10]:
print(type(x))

<class 'torch.Tensor'>


In [11]:
# Load test data

# Load into dataframe
df = pd.read_csv('./wordsim353/combined.csv')
print(df.head())

     Word 1    Word 2  Human (mean)
0      love       sex          6.77
1     tiger       cat          7.35
2     tiger     tiger         10.00
3      book     paper          7.46
4  computer  keyboard          7.62


In [12]:
# Get word embeddings
sample_embedding = get_word_embedding(model ,df['Word 1'][1])
print(sample_embedding.shape)
sample_embedding = sample_embedding.squeeze()
print(sample_embedding.shape)
print(sample_embedding)

(300,)
(300,)
[-0.010743   -0.0084354   0.092938   -0.01284    -0.021971   -0.0032322
  0.038068   -0.10577     0.056288    0.0030197   0.05506     0.016761
  0.10294     0.02478    -0.035468   -0.0082909   0.073924   -0.20287
  0.056451    0.010568    0.04613    -0.028164   -0.010605    0.010803
 -0.0011941   0.039338    0.018323   -0.00094558  0.039761    0.022547
  0.021721    0.0021049   0.092333    0.078911   -0.0025022  -0.025226
  0.033931   -0.083757    0.037918    0.036854    0.040416   -0.11903
  0.087746   -0.038085    0.025236    0.037645    0.10565     0.011185
  0.02687     0.046986    0.04401     0.071783   -0.0093949  -0.088606
 -0.0076906  -0.11334     0.046087   -0.062826   -0.10398     0.067696
  0.077696   -0.091606    0.058478   -0.0049788   0.028417    0.049761
  0.015129    0.0077711   0.06533     0.00049186  0.032615   -0.10434
  0.032913   -0.086249    0.029265    0.054921   -0.045848   -0.033614
  0.034103   -0.05848     0.0074672  -0.064005   -0.015384    0.0

In [13]:
# Check similarity between two words
word1 = df['Word 1'][1]
word2 = df['Word 1'][1]
# Use gensim matutils to calculate cosine similarity
w1 = model[word1]
w2 = model[word2]

print(type(w1))

sim = dot(matutils.unitvec(w1), matutils.unitvec(w2))
print(sim)

<class 'numpy.ndarray'>
1.0


In [14]:
# Function to get cosine similarity
def cos_similarity(word1_embedding, word2_embedding):
    ans = dot(matutils.unitvec(word1_embedding), matutils.unitvec(word2_embedding))
    return ans

# Function to get Pearson correlation
def pearson_correlation(word1_embedding, word2_embedding):
    emb1 = np.array(word1_embedding)
    emb2 = np.array(word2_embedding)

    correlation, _ = pearsonr(emb1, emb2)
    return correlation

In [17]:
def test_sim(df, lemmatizer, stemmer):
    cosine_similarity_scores = []
    pearson_correlation_scores = []
    scores = []

    for _, row in df.iterrows():
        word1 = row['Word 1']
        word2 = row['Word 2']
        
        # Get embeddings
        word1_embedding = get_word_embedding(model,word1).squeeze()
        word2_embedding = get_word_embedding(model,word2).squeeze()

        # Get cosine similarity
        cosine_similarity_scores.append(cos_similarity(word1_embedding, word2_embedding))
        
        # Get pearson correlation
        pearson_correlation_scores.append(pearson_correlation(word1_embedding, word2_embedding))

        # Get score
        scores.append(row['Human (mean)'])
        
    return cosine_similarity_scores, pearson_correlation_scores, scores

In [18]:
# Get cosine similarity and pearson correlation scores
lemmatizer = WordNetLemmatizer()
stemmer = nltk.stem.PorterStemmer()
cosine_similarity_scores, pearson_correlation_scores, scores = test_sim(df, lemmatizer, stemmer)

In [19]:
# Check cosine similarity and pearson correlation scores
print(type(cosine_similarity_scores))
print(type(pearson_correlation_scores))
print(type(scores))

<class 'list'>
<class 'list'>
<class 'list'>


### Initial Spearman

In [20]:
# Funtcion to get spearman correlation using cosine similarity scores
def spearman_correlation(cosine_similarity_scores, simlex_scores):
    # Scale cosine similarity scores to 0-10
    cosine_similarity_scores = np.array(cosine_similarity_scores)
    cosine_similarity_scores = (1+cosine_similarity_scores)*5
    # cosine_similarity_scores *= 10
    simlex_scores = np.array(simlex_scores)

    correlation, _ = spearmanr(cosine_similarity_scores, simlex_scores)
    return correlation

In [21]:
# Print the initial spearman correlation
spearman_value_sim = spearman_correlation(cosine_similarity_scores, scores)
print("Initial Spearman correlation: ", spearman_value_sim)

Initial Spearman correlation:  0.6943215016552395
